# Notities

## Handige links
* [Statline portaal](https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS)
* [Snelstartgids open data](https://www.cbs.nl/nl-nl/onze-diensten/open-data/statline-als-open-data/snelstartgids)
* [repo Github voor cbsodata](https://github.com/J535D165/cbsodata)
* [PyPi documentatis cbsodata](https://pypi.org/project/cbsodata/)
* [Sphinx docs cbsodata](https://cbsodata.readthedocs.io/en/latest/index.html)
* [GitHub voorbeelden OData 3](https://github.com/statistiekcbs/CBS-Open-Data-v3)
* [GitHub voorbeelden OData 4](https://github.com/statistiekcbs/CBS-Open-Data-v4/tree/master/Python)
* [Statline voor derden](https://www.cbs.nl/nl-nl/cijfers/statline/statline-voor-derden)
* [CBS open data Statline](https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS)
* [PDOK Viewer: Kies bij dataset voor CBS gebiedsindelingen (wijken staan helemaal onderaan) voor kaartweergave](https://www.pdok.nl/viewer/)

# Imports

In [ ]:
import pandas as pd
import cbsodata

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import altair as alt
from sklearn import preprocessing


# Settings

In [ ]:
GEMEENTES_REGIO_NIJMEGEN = ['Berg en Dal', 'Beuningen', 'Druten', 'Heumen', 'Mook en Middelaar', 'Nijmegen', 'Wijchen']

KEYWORDS = ['wmo', 'gemeente', 'regio', 'postcode', 'zorg', 'werk', 'lonen', 'loon', 'inkomen', 'conjunctuur', 
            'prognose', 'afstand', 'wijk', 'voorziening', 'genees', 'gezond']
URLS = ['opendata.cbs.nl', 'dataderden.cbs.nl']

CBS_OPEN_URL = 'opendata.cbs.nl'
WMO_TABLES = {'2020': '84907NED',
            '2019': '84664NED',
            '2018': '84421NED',
            '2017': '83818NED',
            '2016': '83620NED',
            '2015': '83267NED'}
WIJK_TABLES = {'2020': '84799NED', 
               '2019': '84583NED', 
               '2018': '84286NED', 
               '2017': '83765NED', 
               '2016': '83487NED', 
               '2015': '83220NED'} 
#                '2014': '82931NED'}

DICT_REALTIVELY_COLS = {'aantalinwoners': ['percentagewmoclienten', 'mannen', 'vrouwen', 'k0tot15jaar', 'k15tot25jaar',
                                           'k25tot45jaar', 'k45tot65jaar', 'k65jaarofouder', 'ongehuwd', 'gehuwd', 'gescheiden',
                                           'verweduwd', 'westerstotaal', 'nietwesterstotaal', 'marokko', 
                                           'nederlandseantillenenaruba', 'suriname', 'turkije', 'overignietwesters', 
                                           'geboortetotaal', 'geboorterelatief', 'sterftetotaal', 'sterfterelatief', 
                                           'aantalinkomensontvangers', 'personenpersoortuitkeringbijstand', 
                                           'personenpersoortuitkeringao', 'personenpersoortuitkeringww', 
                                           'personenpersoortuitkeringaow'],
                        'huishoudenstotaal': ['eenpersoonshuishoudens', 'huishoudenszonderkinderen', 'huishoudensmetkinderen'],
                        'bedrijfsvestigingentotaal': ['alandbouwbosbouwenvisserij', 'bfnijverheidenenergie', 
                                                      'gihandelenhoreca', 'hjvervoerinformatieencommunicatie', 
                                                      'klfinancieledienstenonroerendgoed', 'mnzakelijkedienstverlening', 
                                                      'rucultuurrecreatieoverigediensten'],
                        'personenautostotaal': ['personenautosbrandstofbenzine', 'personenautosoverigebrandstof'],
                        'oppervlaktetotaal': ['oppervlakteland', 'oppervlaktewater']}
LIST_NORM_COLS = ['wmoclienten', 'aantalinwoners', 'huishoudenstotaal', 'gemiddeldehuishoudensgrootte', 'bevolkingsdichtheid', 
                  'woningvoorraad', 'gemiddeldewoningwaarde', 'percentageeengezinswoning', 'percentagemeergezinswoning', 
                  'percentagebewoond', 'percentageonbewoond', 'koopwoningen', 'huurwoningentotaal', 'inbezitwoningcorporatie', 
                  'inbezitoverigeverhuurders', 'eigendomonbekend', 'bouwjaarvoor2000', 'bouwjaarvanaf2000', 
                  'gemiddeldelektriciteitsverbruiktotaal', 'gemelectriciteitsverbruikappartement', 
                  'gemelectriciteitsverbruiktussenwoning', 'gemelectriciteitsverbruikhoekwoning', 
                  'gemelectriciteitsverbruiktweeondereenkapwoning', 'gemelectriciteitsverbruikvrijstaandewoning', 
                  'gemelectriciteitsverbruikhuurwoning', 'gemelectriciteitsverbruikeigenwoning', 
                  'gemiddeldaardgasverbruiktotaal', 'gemgasverbruikappartement', 'gemgasverbruiktussenwoning', 
                  'gemgasverbruikhoekwoning', 'gemgasverbruiktweeondereenkapwoning', 'gemgasverbruikvrijstaandewoning', 
                  'gemgasverbruikhuurwoning', 'gemgasverbruikeigenwoning', 'percentagewoningenmetstadsverwarming',
                  'gemiddeldinkomenperinkomensontvanger', 'gemiddeldinkomenperinwoner', 'k40personenmetlaagsteinkomen', 
                  'k20personenmethoogsteinkomen', 'actieven1575jaar', 'k40huishoudensmetlaagsteinkomen', 
                  'k20huishoudensmethoogsteinkomen', 'huishoudensmeteenlaaginkomen', 'huishonderofrondsociaalminimum',
                  'bedrijfsvestigingentotaal', 'personenautostotaal', 'motorfietsen', 'afstandtothuisartsenpraktijk', 
                  'afstandtotgrotesupermarkt', 'afstandtotkinderdagverblijf', 'afstandtotschool', 'scholenbinnen3km',
                  'oppervlaktetotaal', 'matevanstedelijkheid', 'omgevingsadressendichtheid']
DROP_COLS = ['financieringsvorm', 'wmoclientenper1000inwoners', 'gemeentenaam', 'meestvoorkomendepostcode', 'dekkingspercentage', 'totaaldiefstaluitwoningschuured', 
               'vernielingmisdrijftegenopenbareorde', 'geweldsenseksuelemisdrijven', 'personenautosjongerdan6jaar', 
               'personenautos6jaarenouder', 'bedrijfsmotorvoertuigen']

# Functies

In [ ]:
def make_cols_relatively(df, dict_relatively_cols):
    for base_col, relatively_cols in dict_relatively_cols.items():
        df[relatively_cols] = df[relatively_cols].div(df[base_col], axis=0)
    return df

def normalize_cols(df, norm_cols):
    df_sub = df.copy()
    df_sub = df_sub[norm_cols]
    x = df_sub.values #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df_sub=pd.DataFrame(x_scaled, columns=df_sub.columns, index=df_sub.index)
    df = df.drop(norm_cols, axis=1).join(df_sub)
    return df

#  Load dataset

In [ ]:
df_dataset_WMO = pd.read_parquet('../data/df_dataset_WMO.parquet.gzip')
print(f"Shape of dataset: {df_dataset_WMO.shape}")
df_dataset_WMO.sample(5)

In [ ]:
df_dataset_WMO.shape

## Ways to get a subset of the DataFrame:

In [ ]:
# Subset on columnvalue:
df_dataset_WMO[df_dataset_WMO['gemeentenaam']=='Nijmegen']

# One row / record
df_dataset_WMO.loc[('WK026801', '2019')]

# Multiple rows / records based on combination of the multiindex
df_dataset_WMO.loc[[('WK026801', '2018'), ('WK026802', '2018')]]

# Multiple rows / records for one column (works only for series)
df_dataset_WMO['gemeentenaam'].loc[(['WK026801', 'WK026802'], ['2018', '2019'])]

# Multiple rows based on both indexes:
df_dataset_WMO.loc(axis=0)[['WK026801', 'WK026802'], ['2018', '2019']]

# Subset with IndexSlice
idx = pd.IndexSlice
# Subset on one of the multiindex and select a column
df_dataset_WMO.loc[idx[:, ['2018', '2019']], idx["gemeentenaam"]]
# Subset on one of the multiindex and select all columns
df_dataset_WMO.loc[idx['WK026801', :], idx[:]]

## Transform data

In [ ]:
df = df_dataset_WMO.copy()
df['percentagewmoclienten'] = df['wmoclienten']
df = df.drop(DROP_COLS, axis=1)
df = make_cols_relatively(df=df, dict_relatively_cols=DICT_REALTIVELY_COLS)
df = normalize_cols(df=df, norm_cols=LIST_NORM_COLS)

In [ ]:
df.shape

# EDA data

In [ ]:
df_dataset_WMO.head()

In [ ]:
df_dataset_WMO['wmoclienten'].plot()

In [ ]:
df_dataset_WMO['wmoclientenper1000inwoners'].plot()

In [ ]:
df_dataset_WMO[df_dataset_WMO['wmoclientenper1000inwoners']==1000]

In [ ]:
df_corr_with_na = df.copy().corr()

In [ ]:
df_corr = df_corr_with_na.dropna(axis=1, how='all').dropna(axis=0, how='all')

In [ ]:
df_corr = df_corr.fillna(0)

In [ ]:
df_corr.shape

In [ ]:
df_corr

In [ ]:
# https://towardsdatascience.com/altair-plot-deconstruction-visualizing-the-correlation-structure-of-weather-data-38fb5668c5b1
cor_data = df_corr.stack().reset_index().rename(columns={0: 'correlation', 'level_0': 'variable', 'level_1': 'variable2'})
cor_data['correlation_label'] = cor_data['correlation'].map('{:.2f}'.format)
cor_data.head()

In [ ]:
cor_data.shape

In [ ]:
cor_data[cor_data['variable']=='percentagewmoclienten']

In [ ]:
cor_data_sub = cor_data[(cor_data['correlation']<-0.7)|(cor_data['correlation']>0.7)]
cor_data_sub.shape

In [ ]:
base = alt.Chart(cor_data_sub).encode(
    x='variable2:O',
    y='variable:O'    
)

# Text layer with correlation labels
# Colors are for easier readability
text = base.mark_text().encode(
    text='correlation_label',
    color=alt.condition(
        alt.datum.correlation > 0.5, 
        alt.value('white'),
        alt.value('black')
    )
)

# The correlation heatmap itself
cor_plot = base.mark_rect().encode(
    color='correlation:Q'
)

cor_plot + text # The '+' means overlaying the text and rect layer

In [ ]:
df_dataset_WMO.head()

In [ ]:
# source = df_dataset_WMO[['wmoclienten', 'wmoclientenper1000inwoners', 'AantalInwoners_5']].copy().loc(axis=0)[:, ['2019']].reset_index()
# source = source.fillna(-5)
# # source[['wmoclienten', 'wmoclientenper1000inwoners', 'AantalInwoners_5']]

In [ ]:
# source
jaar = '2019'
source = df_dataset_WMO.copy().loc(axis=0)[:, [jaar]].reset_index()
source

In [ ]:
jaar = '2019'
source = df_dataset_WMO.copy().loc(axis=0)[:, [jaar]].reset_index()
alt.Chart(source, title=f"WMO clienten vs WMO clienten per 1000 inwoners voor jaar {jaar}",).mark_circle(size=60).encode(
    x='wmoclienten',
    y='wmoclientenper1000inwoners',
    color='AantalInwoners_5',
    tooltip=['codering_regio', 'perioden', 'gemeentenaam', 'AantalInwoners_5', 'wmoclienten', 'wmoclientenper1000inwoners']
).interactive()

In [ ]:
jaar = '2018'
source = df_dataset_WMO.copy().loc(axis=0)[:, [jaar]].reset_index()
alt.Chart(source, title=f"WMO clienten vs WMO clienten per 1000 inwoners voor jaar {jaar}",).mark_circle(size=60).encode(
    x='wmoclienten',
    y='wmoclientenper1000inwoners',
    color='AantalInwoners_5',
    tooltip=['codering_regio', 'perioden', 'gemeentenaam', 'AantalInwoners_5', 'wmoclienten', 'wmoclientenper1000inwoners']
).interactive()

In [ ]:
jaar = '2017'
source = df_dataset_WMO.copy().loc(axis=0)[:, [jaar]].reset_index()
alt.Chart(source, title=f"WMO clienten vs WMO clienten per 1000 inwoners voor jaar {jaar}",).mark_circle(size=60).encode(
    x='wmoclienten',
    y='wmoclientenper1000inwoners',
    color='AantalInwoners_5',
    tooltip=['codering_regio', 'perioden', 'gemeentenaam', 'AantalInwoners_5', 'wmoclienten', 'wmoclientenper1000inwoners']
).interactive()

In [ ]:
jaar = '2016'
source = df_dataset_WMO.copy().loc(axis=0)[:, [jaar]].reset_index()
alt.Chart(source, title=f"WMO clienten vs WMO clienten per 1000 inwoners voor jaar {jaar}",).mark_circle(size=60).encode(
    x='wmoclienten',
    y='wmoclientenper1000inwoners',
    color='AantalInwoners_5',
    tooltip=['codering_regio', 'perioden', 'gemeentenaam', 'AantalInwoners_5', 'wmoclienten', 'wmoclientenper1000inwoners']
).interactive()

In [ ]:
# for jaar in ['2016', '2017', '2018', '2019']:
#     source = df_dataset_WMO.copy().loc(axis=0)[:, [jaar]].reset_index()
#     alt.Chart(source, title=f"WMO clienten vs WMO clienten per 1000 inwoners voor jaar {jaar}",).mark_circle(size=60).encode(
#         x='wmoclienten',
#         y='wmoclientenper1000inwoners',
#         color='AantalInwoners_5',
#         tooltip=['AantalInwoners_5', 'wmoclienten', 'wmoclientenper1000inwoners']
#     ).interactive()

# Klad

In [ ]:
# # DRAAIT LANG ALS JE ALLES OPVRAAGT
# url = 'dataderden.cbs.nl'
# df_to_collect = df_tables_cbs_derden
# dict_cbs_tables = {}
# for i in df_to_collect['Identifier'][:2]:
#     print(i)
#     try:
#         df = pd.DataFrame(cbsodata.get_data(str(i), catalog_url=url))
#     except Exception:
#         df = pd.DataFrame()
#         pass
#     dict_cbs_tables[i] = df
# dict_cbs_tables['40072NED'].sample(10)

In [ ]:
# # Eerste lijst met mogelijke tabellen
# dict_tables = {'regio-indeling 2021': '84929NED',
#                'regio-indeling 2020':'84721NED', 
#                'regio-indeling 2019':'84378NED', 
#                'regio-indeling 2018':'83859NED', 
#                'regio-indeling 2017':'83553NED', 
#                'regio-indeling 2016':'83287NED',
#                 'Bevolking; geslacht, leeftijd, regio': '37259ned',
#                 'Huishoudens; samenstelling, regio': '71486ned',
#                 'Personen; afstand tot ouder, 2014': '83246NED',
#                 'Bevolking; nationaliteit en regio' : '84727NED',
#                 'Bevolking; leeftijd, regio, 2020-2050': '84525NED',
#                 'Bevolkingsontwikkeling, regio, 2020-2050': '84528NED',
#                 'doodsoorzaken':'80202ned',
#                 'Geneesmiddelen; regio (gemeente)': '83251NED',
#                 'DBC-GGZ; personen per diagnose; regio': '84859NED',
#                 'Indicatoren jeugdzorg; gemeenten': '82972NED',
#                 'Jeugdzorgcombinaties': '84136NED',
#                 'Jongeren met jeugdzorg; pers. kenmerken':'84135NED',
#                 'Trajecten jeugdzorg; regio, peildatum': '82968NED',
#                 'Gemeentelijke kosten; jeugdzorg, regio': '83454NED',
#                 'Wmo-cliënten; financiering, wijken, 2020': '84907NED',
#                 'Wmo-cliënten; financiering, wijken, 2019': '84664NED',
#                 'Wmo-cliënten; financiering, wijken, 2018': '84421NED',
#                 'Wmo-cliënten; financiering, wijken, 2017': '83818NED',
#                 'Wmo-cliënten; financiering, wijken 2016': '83620NED',
#                 'Wmo-cliënten; financiering, wijken 2015':'83267NED',
#                 'Wmo_cliënten; type, wijken, 2020': '84908NED',
#                 'Wmo-cliënten; type maatwerk, wijken 2019': '84753NED',
#                 'Wmo-cliënten; type maatwerk, wijken 2018': '84752NED',
#                 'Wmo-cliënten; type maatwerk, wijken 2017': '84751NED',
#                 'Inkomsten eigen bijdr. Wmo-maatwerkvoorz': '84578NED',
#                 'Uitgaven Wmo-maatwerkvoorz.; type, regio': '84580NED',
#                 'Wmo-cliënten die instromen in Wlz 2019': '84812NED',
#                 'Wmo-cliënten die instromen in Wlz 2018': '84599NED',
#                 'Wmo-cliënten die instromen in Wlz 2017': '84579NED',
#                 'Wmo-arrangementen; stand, regio': '83268NED',
#                 'Wmo-cliënten; reden beëindiging, regio': '83266NED',
#                 'Wmo-cliënten; type arrangement, regio':'83262NED',
#                 'Wmo-cliënten; type maatwerk; regio': '84839NED',
#                 'Wmo-gebruik; aantal arrangementen, regio': '83264NED',
#                 'Stapeling Sociaal Domein, 2020': '84906NED',
#                 'Gebruik voorz sociaal dom; wijken, 2019': '84662NED',
#                 'Gebr voorz sociaal domein; wijken, 2018': '84420NED',
#                 'Gebr voorz sociaal domein; wijken, 2017': '83817NED',
#                 'Gebr voorz sociaal domein; wijken 2016': '83619NED',
#                 'Inkomen huishoudens; kenmerken, regio': '84866NED',
#                 'Kerncijfers begrot. heffing per gemeente': '83643NED',
#                 'Gemeenterek.; taakvelden groottekl regio': '84413NED',
#                 'Gem.rekening; balans regio grootteklasse': '71231ned'}

In [ ]:
# # Crasht wegens memory problemen
# df_meta = pd.DataFrame()
# for table in dict_tables.values():
#     df = pd.DataFrame(cbsodata.get_data(table, catalog_url=url))
#     if 'RegioS' in df.columns:
#         regios = df.RegioS.unique()
#     else:
#         regios = "NOTPRESENT"
#     if 'Periode' in df.columns:
#         periode = df.Periode.unique()
#     elif 'Perioden' in df.columns:
#         periode = df.Perioden.unique()
#     else:
#         periode = "NOTPRESENT"
#     df.to_csv(f"{table}.csv")
#     df_meta = df_meta.append({'table':table, 'regios':regios, 'periode':periode}, ignore_index=True)
# df_meta.to_csv('df_meta_long.csv', sep=';')

In [ ]:
# # Get meta data
# table = '84727NED'
# print(cbsodata.get_info(table)['ShortDescription'])
# meta_data = pd.DataFrame(cbsodata.get_meta(table, "DataProperties"))
# meta_data.head(10)

In [ ]:
# # tabel te groot
# table = '84727NED'
# df = pd.DataFrame(cbsodata.get_data(table, catalog_url=url))

In [ ]:
# df_84799NED = pd.DataFrame(cbsodata.get_data('84799NED', catalog_url=CBS_OPEN_URL))
# df_84583NED = pd.DataFrame(cbsodata.get_data('84583NED', catalog_url=CBS_OPEN_URL))
# df_84286NED = pd.DataFrame(cbsodata.get_data('84286NED', catalog_url=CBS_OPEN_URL))
# df_83765NED = pd.DataFrame(cbsodata.get_data('83765NED', catalog_url=CBS_OPEN_URL))
# df_83487NED = pd.DataFrame(cbsodata.get_data('83487NED', catalog_url=CBS_OPEN_URL))
# df_83220NED = pd.DataFrame(cbsodata.get_data('83220NED', catalog_url=CBS_OPEN_URL))
# df_82931NED = pd.DataFrame(cbsodata.get_data('82931NED', catalog_url=CBS_OPEN_URL))